# 第一章：基础提示结构

- [课程内容](#lesson)
- [练习](#exercises)
- [示例练习场](#example-playground)

## 设置

运行以下设置单元格来加载您的 API 密钥并建立 `get_completion` 辅助函数。

In [15]:
# 导入 Python 内置的正则表达式库
import re
from openai import OpenAI

# 从 IPython 存储中检索 API_KEY 和 MODEL_NAME 变量
%store -r API_KEY
%store -r MODEL_NAME
%store -r BASE_URL

client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

def get_completion(prompt: str, system_prompt=""):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    
    response = client.chat.completions.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=messages
    )
    return response.choices[0].message.content

---

## 课程内容

OpenAI 提供了多种 API 接口，在本教程中我们将使用 [Chat Completions API](https://platform.openai.com/docs/api-reference/chat)。

使用 Chat Completions API 调用模型至少需要以下参数：
- `model`: 您要调用的[模型名称](https://platform.openai.com/docs/models)

- `max_tokens`: 在停止之前生成的最大令牌数。请注意，模型可能在达到此最大值之前停止。此参数仅指定要生成的绝对最大令牌数。此外，这是一个*硬*停止，意味着它可能导致模型在单词或句子中间停止生成。

- `messages`: 输入消息数组。我们的模型被训练为在交替的 `user` 和 `assistant` 对话轮次上运行。创建新消息时，您使用 messages 参数指定先前的对话轮次，然后模型生成对话中的下一条消息。
  - 每条输入消息必须是具有 `role` 和 `content` 的对象。您可以指定单个 `user` 角色消息，也可以包含多个 `user` 和 `assistant` 消息（如果是这样，它们必须交替出现）。第一条消息必须始终使用 `user` 角色。

还有一些可选参数，例如：
- `system`: 系统提示 - 下面会详细介绍。
  
- `temperature`: 模型响应的变化程度。在这些课程和练习中，我们将 `temperature` 设置为 0。

有关所有 API 参数的完整列表，请访问我们的 [API 文档](https://platform.openai.com/docs/api-reference/chat)。

### 示例

让我们看看模型如何响应一些格式正确的提示。对于以下每个单元格，运行单元格（`shift+enter`），模型的响应将出现在代码块下方。

In [ ]:
# 提示
PROMPT = "你好，AI助手，你好吗？"

# 打印模型的响应
print(get_completion(PROMPT))

In [ ]:
# 提示
PROMPT = "你能告诉我海洋的颜色吗？"

# 打印模型的响应
print(get_completion(PROMPT))

In [ ]:
# 提示
PROMPT = "席琳·迪翁是哪一年出生的？"

# 打印模型的响应
print(get_completion(PROMPT))

现在让我们看一些没有包含正确的 Chat Completions API 格式的提示。对于这些格式错误的提示，API 会返回错误。

首先，我们有一个在 `messages` 数组中缺少 `role` 和 `content` 字段的 API 调用示例。

In [ ]:
# 获取模型的响应
response = client.chat.completions.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"你好，AI助手，你好吗？"}
        ]
    )

# 打印模型的响应
print(response.choices[0].message.content)

`user` 和 `assistant` 消息**必须交替出现**，消息**必须以 `user` 轮次开始**。您可以在提示中有多个 `user` 和 `assistant` 对（就像模拟多轮对话一样）。您还可以在终端 `assistant` 消息中放置文字，让模型从您停下的地方继续（在后面的章节中会详细介绍）。

#### 系统提示

您还可以使用**系统提示**。系统提示是在"用户"轮次中向模型提出问题或任务之前，**为模型提供上下文、指令和指导**的一种方式。

在结构上，系统提示独立于 `user` 和 `assistant` 消息列表之外，因此属于单独的 `system` 参数（查看笔记本[设置](#setup)部分中 `get_completion` 辅助函数的结构）。

在本教程中，无论我们在哪里可能使用系统提示，我们都在完成函数中为您提供了 `system` 字段。如果您不想使用系统提示，只需将 `SYSTEM_PROMPT` 变量设置为空字符串即可。

#### 系统提示示例

In [ ]:
# 系统提示
SYSTEM_PROMPT = "你的回答应该始终是一系列批判性思维问题，以推进对话的发展（不要提供问题的答案）。不要直接回答用户的问题。"

# 提示
PROMPT = "为什么天空是蓝色的？"

# 打印模型的响应
print(get_completion(PROMPT, SYSTEM_PROMPT))

为什么要使用系统提示？**编写良好的系统提示可以通过多种方式提高模型的性能**，比如增强模型遵循规则和指令的能力。有关更多信息，请访问我们关于[如何使用系统提示](https://platform.openai.com/docs/guides/text-generation/system-messages)的文档。

现在我们将深入一些练习。如果您想在不更改上述任何内容的情况下试验课程提示，请滚动到课程笔记本的最底部访问[**示例练习场**](#example-playground)。

---

## 练习
- [练习 1.1 - 数数到三](#exercise-11---counting-to-three)
- [练习 1.2 - 系统提示](#exercise-12---system-prompt)

### 练习 1.1 - 数数到三
使用正确的 `user` / `assistant` 格式，编辑下面的 `PROMPT` 来让模型**数数到三**。输出还会显示您的解决方案是否正确。

In [ ]:
# 提示 - 这是您应该更改的唯一字段
PROMPT = "[替换此文本]"

# 获取模型的响应
response = get_completion(PROMPT)

# 用于评分练习正确性的函数
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# 打印模型的响应和相应的评分
print(response)
print("\n--------------------------- 评分 ---------------------------")
print("这个练习已正确解决:", grade_exercise(response))

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
from hints import exercise_1_1_hint; print(exercise_1_1_hint)

### 练习 1.2 - 系统提示

修改 `SYSTEM_PROMPT` 让模型像 3 岁小孩一样回应。

In [ ]:
# 系统提示 - 这是您应该更改的唯一字段
SYSTEM_PROMPT = "[替换此文本]"

# 提示
PROMPT = "天空有多大？"

# 获取模型的响应
response = get_completion(PROMPT, SYSTEM_PROMPT)

# 用于评分练习正确性的函数
def grade_exercise(text):
    return bool(re.search(r"咯咯笑", text) or re.search(r"好大好大", text) or re.search(r"超级", text) or re.search(r"哇", text))

# 打印模型的响应和相应的评分
print(response)
print("\n--------------------------- 评分 ---------------------------")
print("这个练习已正确解决:", grade_exercise(response))

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
from hints import exercise_1_2_hint; print(exercise_1_2_hint)

### 恭喜！

如果您已经解决了到目前为止的所有练习，您就可以进入下一章了。愉快的提示工程学习！

---

## 示例练习场

这是一个供您自由试验本课程中展示的提示示例并调整提示以查看如何影响模型响应的区域。

In [ ]:
# 提示
PROMPT = "你好，AI助手，你好吗？"

# 打印模型的响应
print(get_completion(PROMPT))

In [ ]:
# 提示
PROMPT = "你能告诉我海洋的颜色吗？"

# 打印模型的响应
print(get_completion(PROMPT))

In [ ]:
# 提示
PROMPT = "席琳·迪翁是哪一年出生的？"

# 打印模型的响应
print(get_completion(PROMPT))

In [ ]:
# 获取模型的响应
response = client.chat.completions.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"你好，AI助手，你好吗？"}
        ]
    )

# 打印模型的响应
print(response.choices[0].message.content)

In [ ]:
# 获取模型的响应
response = client.chat.completions.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "席琳·迪翁是哪一年出生的？"},
          {"role": "user", "content": "另外，你能告诉我一些关于她的其他事实吗？"}
        ]
    )

# 打印模型的响应
print(response.choices[0].message.content)

In [ ]:
# 系统提示
SYSTEM_PROMPT = "你的回答应该始终是一系列批判性思维问题，以推进对话的发展（不要提供问题的答案）。不要直接回答用户的问题。"

# 提示
PROMPT = "为什么天空是蓝色的？"

# 打印模型的响应
print(get_completion(PROMPT, SYSTEM_PROMPT))